<img src="./img/Logo.jpg" align="right" width="150" />

## Medidas Electrónicas 1

# Trabajo Práctico de Laboratorio Nº 8 : Método Diferencial

### **Curso:**
* R4052 2023

### **Docentes:**
* Marinsek, Emiliano
* Perdomo, Juan Manuel

### **Nº de Grupo:**
* 2
###   **Integrantes:**
* Borello, Federico
* Dimaio, Agustin
* Manoukian, Francisco Tomas
* Mella, Camila
* Ronchetti Juan Manuel

# Objetivo

Realizar la medición de regulación de carga de un regulador de tensión `7805`.

Ecuación general del método diferencial:
$$ V_i = V_2 - V_1$$

Circuito Propuesto:

<img src="./img/TPL8 - Metodo Diferencial (2023) - R4052 -.svg" width="650" />


## Procedimiento

1. Con la llave abierta, se ajusta el potenciometro hasta lograr que la tensión indicada por el voltímetro sea igual, o lo más cercana a cero posible. En este paso se obtiene $V_1$.
2. Cerrar la llave y tomar la medición que indica el voltímetro. En este paso se obtiene $V_2$.
3. Abrir la llave y verificar que la indicación del voltímetro vuelva a cero o a un valor cercano al que se tomó en el paso 1. Si no es el caso, se descarta la medición. En este paso se obtiene $V_3$

>- NOTA: Solamente se utilizan $V_2$ y $V_1$. $V_3$ es una verificación.

El paso 3 se realiza por el hecho de "bajadas" de tensión que puede sufir la bateria debido a la descarga de la misma.

- Se utiliza una pila de 9V nueva, la cual cuenta con una tensión de `9.54`.
- $R_1 = 33[k\Omega]$
- $R_2 = 25[k\Omega]$ (Potenciómetro)
- $R_1 = 47[k\Omega]$

Con estos valores, el potenciómetro puede proporcionar valores de tensión entre 4V hasta 6V aproximadamente. Lo cual resulta ideal, ya que el 7805 tiene una tensión de salida de 5V.

## Tabla Con Mediciones

In [1]:
import pandas as pd

tabla = pd.read_excel("./Mediciones/TP8.xlsx", nrows=3,  usecols= 'A:E')

# Convierte columnas a list
V_1 = tabla['Vnl_1 [mV]'].tolist()
V_2 = tabla['Vl [mV]'].tolist()
V_3 = tabla['Vnl_2 [mV]'].tolist()

tabla

,Medicion,Vnl_1 [mV],Vl [mV],IL [A],Vnl_2 [mV]
0,1,0.55,4.35,1.046,0.95
1,2,0.58,5.55,1.060,0.46
2,3,0.01,4.98,1.041,-0.02


# Cálculo de Incertidumbre

- Modelo: UT33C
- Rango: 200mV
- Resolución: 0.01V
- Especificaciones Error Tension: $0.7\% + 3c$
- Display: $3  \frac{1}{2}$
- $R_{\text{in}} = 10[M\Omega]$

In [ ]:
import numpy as np
import sympy as sp
from IPython.display import display, Math

def uA(x) -> float:
    return (np.var(x, ddof=1)/(x.size))**(1/2)

def uB(x, err, dig, c) -> float:
    return ((err/100+dig/c)*np.average(x))/3**(1/2)

def uC(uB, uA) -> float:
    return (uA**2 + uB**2)**(1/2)

def all(x, err, dig, c) -> (float):
    A = uA(x)
    B = uB(x,err,dig,c)
    C = uC(A, B)

    return (A, B, C)


In [ ]:
#Mediciones
V1 = np.array(V_1)
V2 = np.array(V_2)
#Especificaciones:
errV = 0.7
dig = 3

#Cuentas
uA_V1, uB_V1, uC_V1 = all(V1, errV, dig, 38)
uA_V2, uB_V2, uC_V2 = all(V2, errV, dig, 496)

# Se visualiza el valor medio para saber las cuentas.
# También se podrian obtener las cuentas dividiendo el valor por la escala
print(f"V1_avg: {np.average(V1)}")
print(f"V2_avg: {np.average(V2)}")

V1_avg: 0.37999999999999995
V2_avg: 4.96


In [ ]:
# Symbolic derivation
v_1, v_2 = sp.symbols('V_1 V_2')
V = v_2 - v_1

V_avg = V.subs([(v_1, np.average(V1)), (v_2, np.average(V2))])

uC_V = ( sp.diff(V, v_1)**2 * uC_V1**2 + sp.diff(V, v_2)**2 * uC_V2**2)**(1/2)
uC_V_subs = uC_V.subs([(v_1, np.average(V1)), (v_2, np.average(V2))])
k = 2

display(Math(fr'\begin{{equation}} V_{{i}} = {sp.latex(V)} \quad || \quad \bar{{V_i}} = {round(V_avg, 2)} \; [mV] \nonumber \end{{equation}}'))
display(Math(fr'\begin{{equation}} u_{{C}}(\bar{{V_i}}) = {round(uC_V_subs, 5)}  \; [mV] \nonumber \end{{equation}}'))
display(Math(fr'\begin{{equation}} U(\bar{{V_i}}) = u_C(\bar{{V_i}}) \; k_{{95\%}} \Rightarrow  {round(uC_V_subs * k, 5)} \; [mV] \nonumber \end{{equation}}'))
display(Math(fr'\begin{{equation}} V_B = {format(V_avg, ".3g")} \pm {round(uC_V_subs * 2, 2)} \; [mV]\nonumber \end{{equation}}'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Se utilizó como regulador de tensión un `7805CV` &rarr; [datasheet](https://html.alldatasheet.es/html-pdf/22676/STMICROELECTRONICS/7805CV/9745/6/7805CV.html)

<img src="./img/7805CV.png" width="650" />

El valor máximo de load regulation es de $100[mV]$. En nuestras mediciones da un valor de $5[mV]$, ubicado por debajo del máximo.

# Conclusiones y Extras

Al realizar las mediciones se puedo apreciar lo difícil que puede llegar a ser conseguir una tensión igual a cero indicada por el voltímetro. Se probó conectando la carga de forma errónea, es decir, no tan cerca de los terminales del regulador de tensión, pero no se apreciaron grandes diferencias. Al conectar la carga lo más cerca posible del regulador a través de cables soldados a las patas, se suprimen posibles caídas de tensión debido a parámetros parásitos de un protoboard.

Luego se probó el corte por temperatura, se incrementó la carga reduciendo el valor de la resistencia de carga para tener mayor corriente y se aprecia como la tensión disminuye a medida que aumenta la temperatura.

Por último, como conclusión, resulta un muy buen valor de regulación de carga $5[mV]$. Para verificar el resultado, hubiera sido ideal medir la regulación de carga con el circuito del transistor, pulso y osciloscopio.